<a href="https://colab.research.google.com/github/thabetAljbreen/RepoT5/blob/main/Arabic_classification_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'arabic-100k-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F543826%2F992592%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240905%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240905T075939Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da7e7c7e3204fba15a731abdcd3f12695088d3c2e8a0a354622e0a2c1dc4f3a63dbf56c7be4aa9a4460d2271c1ca832ddb2078e0eb02f9781e5351bb8154adcf2dfde02997dafcc53470fa91dce52776adde7665873d86792c8bb82dfece7d5717eb5d44f667c778bb45c797ba77670fccb0a63c747876d940cf0d8fabd337a768530042bb3af0cdf7dedb4285d6528a4c99e99b0eb513f7d00362639395e65f120cba36c1a6992f7a935d29be518f9b1045ead21062f7484b1e2eb9a2b6124e716ac94544829270befa55cd2c61c6bb4c38d01f47af732cfcbd826421f8fedb72aaa431c42a572ccb0cdbec5331b221efd9d3046b9027ce3ca8893248446ee6c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 17871683 bytes downloaded
Downloaded and uncompressed: arabic-100k-reviews
Data source import complete.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/kaggle/input/arabic-100k-reviews/ar_reviews_100k.tsv',sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [4]:
df.isnull().sum()

,0
label,0
text,0


In [5]:
df['label'].value_counts()

,count
label,
Positive,33333
Mixed,33333
Negative,33333


In [6]:
df = df[df['label']!='Mixed']

In [7]:
df.duplicated().sum()

0

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words('arabic')))
print(stop_words)

['فاء', 'جنيه', 'وُشْكَانَ', 'أيضا', 'عدا', 'مليم', 'اثنين', 'ثلاثمئة', 'بمن', 'سمعا', 'سبعة', 'علم', 'كأي', 'ثالث', 'فإن', 'ظاء', 'صاد', 'إليكَ', 'عشرون', 'أقل', 'نا', 'أيّان', 'ثمان', 'أخبر', 'ثلاثة', 'جير', 'أ', 'إذن', 'عما', 'ثمّة', 'لن', 'و', 'إما', 'هيّا', 'باء', 'حمو', 'قلما', 'بغتة', 'أبدا', 'علق', 'إنه', 'أوه', 'أبريل', 'أمسى', 'على', 'إياكما', 'هَاتانِ', 'سبعون', 'مع', 'هذان', 'أصبح', 'سنتيم', 'إذ', 'اللتيا', 'إليكما', 'دينار', 'ئ', 'ذوا', 'أخذ', 'مهما', 'أيلول', 'ظنَّ', 'طاق', 'كأين', 'سابع', 'حبذا', 'ضاد', 'نون', 'لهم', 'غ', 'غادر', 'وما', 'لات', 'كثيرا', 'خمسمائة', 'سقى', 'شباط', 'بسّ', 'ذات', 'واو', 'ألفى', 'ستمائة', 'من', 'هذا', 'أجمع', 'بها', 'ليستا', 'صهٍ', 'كي', 'دون', 'تفعلين', 'إياها', 'سرعان', 'كيفما', 'مئة', 'سوى', 'رُبَّ', 'عدَّ', 'يا', 'شَتَّانَ', 'صراحة', 'أمام', 'أبو', 'إمّا', 'اللواتي', 'كليكما', 'هلا', 'إليكنّ', 'دواليك', 'أو', 'أرى', 'إذا', 'سين', 'جميع', 'أم', 'هَذانِ', 'أفعل به', 'ذا', 'ذي', 'ذ', 'إيه', 'ما أفعله', 'تانِك', 'ذانِ', 'تِه', 'هو', 'بل', 'أجل

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re
import string
import sys
import argparse
from nltk.tokenize import word_tokenize

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations


In [ ]:
def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, '', str(text))
    return text

def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    text = remove_emoji(text)
    text = remove_diacritics(text)
    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text

In [ ]:
df['cleanedtext'] = df['text'].apply(clean_text)

In [ ]:
df.head()

In [ ]:
def process_text(text):
    stemmer = nltk.ISRIStemmer()
    word_list = nltk.word_tokenize(text)
    #stemming
    word_list = [stemmer.stem(w) for w in  word_list]
    return ' '.join(word_list)


In [ ]:
df['cleanedtextnew']=df['cleanedtext'].apply(process_text)

In [ ]:
df.head()

In [ ]:
### split data to train and test
from sklearn.model_selection import train_test_split
x,y=df['cleanedtext'],df['label']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)
print(len(x_train))
print(len(y_train))
print(len(x_test))
print(len(y_test))

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
vectorizer=TfidfVectorizer() #(analyzer='char_wb',ngram_range=(3,5),min_df=0.01,max_df=0.3)

In [ ]:
clf=SVC(kernel='rbf')

In [ ]:
from sklearn.pipeline import make_pipeline
pipe=make_pipeline(vectorizer,clf)

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
y_pred=pipe.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)